# Convert Votes to People

## Turn the votes page into a picked file of people with their:
### 1) Star Votes
### 2) Difficulty Ratings
### 3) Route Ticks
### 4) ToDo List

In [1]:
import pandas as pd
import pickle

***
### Step 1: Import the Pickled Votes dataframe object

In [2]:
def import_votes(filepath):
    votes = pd.read_pickle(filepath)
    return votes

### Step 2: Create a dictionary for people

#### Structure is as follows:
person: {starVotes: [climbUrl, vote],

         diffRatings: [climbUrl, vote],
           
         routeTicks: [{climbUrl: tickInfo}],
           
         toDoList: [climbUrl] }
           

In [3]:
def make_users(votes):
    # Initialize a dictionary for people
    people = {}

    # Get the star ratings
    for i in range(len(votes['starRatings'])):
        try:
            for user in votes['starRatings'][i]:
                # If user not in dictionary, add them
                if user[0] not in people:
                    people[user[0]] = {}
                    people[user[0]]['starRatings'] = {}
                    people[user[0]]['diffRatings'] = {}
                    people[user[0]]['ticks'] = {}
                    people[user[0]]['toDoList'] = []
                # Add the star rating to their dictionary
                people[user[0]]['starRatings'][votes.index[i]]=user[1]
        except:
            continue
            
    # Get the diffRatings
    for i in range(len(votes['difficultyRatings'])):
#         try:
        for user in votes['difficultyRatings'][i]:
            # If user not in dictionary, add them
            if user[0] not in people:
                people[user[0]] = {}
                people[user[0]]['starRatings'] = {}
                people[user[0]]['diffRatings'] = {}
                people[user[0]]['ticks'] = {}
                people[user[0]]['toDoList'] = []
            # Add the diff rating to their dictionary
            people[user[0]]['diffRatings'][votes.index[i]]=user[1]
#         except:
#             continue
            
    # Get the Ticks
    for i in range(len(votes['ticks'])):
        try:
            for user in votes['ticks'][i]['tickList']:
                # If user not in dictionary, add them
                if user not in people:
                    people[user] = {}
                    people[user]['starRatings'] = {}
                    people[user]['diffRatings'] = {}
                    people[user]['ticks'] = {}
                    people[user]['toDoList'] = []
                # Add the tick to their dictionary
                # Case: First instance of the user doing the route is being recorded
                if votes.index[i] in people[user]['ticks']:
                    people[user]['ticks'][votes.index[i]][0] = people[user]['ticks'][votes.index[i]][0]+1
                    people[user]['ticks'][votes.index[i]].append(votes['ticks'][i]['tickList'][user])
                # Case: Non-First instance of user ticking route is being recorded
                if votes.index[i] not in people[user]['ticks']:
                    people[user]['ticks'][votes.index[i]]=[1, votes['ticks'][i]['tickList'][user]]
        except:
            continue
            
    # Get the toDoLists
    for i in range(len(votes['toDoList'])):
        try:
            for user in votes['toDoList'][i].keys():
                # If user not in dictionary, add them
                if user not in people:
                    people[user] = {}
                    people[user]['starRatings'] = {}
                    people[user]['diffRatings'] = {}
                    people[user]['ticks'] = {}
                    people[user]['toDoList'] = []
                # Add the diff rating to their dictionary
                people[user]['toDoList'].append(votes.index[i])
        except:
            continue
    
    # Return the dictionary of people
    return people

# Convert (Python Dict) => (Pandas Dataframe)

In [4]:
def convert_people_to_dataframe(people):
    peopleDF = pd.DataFrame.from_dict(people, orient='index')
    return peopleDF

# Save "People Dataframe" as a pickled object

In [5]:
def save_people_dataframe(people, destPath):
    people.to_pickle(destPath)

# Package into one function

In [6]:
def make_people_from_votes(votes_filepath, people_destPath):
    votes = import_votes(votes_filepath)
    people = make_users(votes)
    people = convert_people_to_dataframe(people)
    save_people_dataframe(people, people_destPath)
    
filepath = './Data/pickledVotes_unpacked'
destPath = './Data/pickledPeople'
make_people_from_votes(filepath, destPath)